# 导入库

In [27]:
import csv
import os
import cv2
import numpy as np
import keras
import scipy.io
import pandas as pd
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.utils import np_utils

from os.path import join as pjoin
from scipy import misc

# 1.将ID和其类别用字典方式存储，方便后面做标签

In [2]:
with open('feats.csv') as f:
    dict = {}
    reader = csv.DictReader(f)
    for row in reader:
        dict[row['id']]=row['molecular_subtype']  #将ID和其所属亚型标签存为字典便于后期使用。

# 2.读取每个ID文件夹名称

In [3]:
filePath = 'F:\\BaiduNetdiskDownload\\cancer_detection_homework\\train\\images' #读取相应目录下文件夹名字
filename = os.listdir(filePath)
print(filename[0])

009a7cbadad5bffef7ad59ddec0c0312a73eb686e8beae3474eb3955407893b7


# 3.将图片以灰度图像读入，归一化，并且存入trainx里（199, 640, 435）（其中有一个文件夹下没有图像，所以是199个ID类别）

In [4]:
trainx = np.zeros((199, 640, 435))
def load_data(data_dir):  # 
    count = 0
    for guy in os.listdir(data_dir):  
        person_dir = pjoin(data_dir, guy)  #
        for i in os.listdir(person_dir):
            image_dir = pjoin(person_dir, i)  # 
            img1 = cv2.imread(image_dir, cv2.IMREAD_GRAYSCALE)   #读取图片，第二个参数表示以灰度图像读入
            if img1 is None:                                     #判断读入的img1是否为空，为空就继续下一轮循环
                continue
            res1= cv2.resize(img1,(435,640))/255                 #对图片进行缩放，第一个参数是读入的图片，第二个是制定的缩放大小
            #print(res1.shape)
            trainx[count,:] = res1
            count = count+1
        print(person_dir)
    print(count)
    return count
load_data('F:\\BaiduNetdiskDownload\\cancer_detection_homework\\train\\images')

F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\009a7cbadad5bffef7ad59ddec0c0312a73eb686e8beae3474eb3955407893b7
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\00b0847ed013c1c8a20e36bd45e7cbf825b5de366ee69cc9a289442919e583e5
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\0101c6663374a5642ddd7797c30be68580aca958778b401c6cd5025c2e521e0c
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\0171068ffcfac22e60db2aa4bd7de00cf63d1e2297981918533f799eaa753a6c
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\02855bc49b3806a38ce5774c575a3e5dd40762e2454b89ef9911c80266c13414
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\044be4072454995a83d90c2856af619a367c05208caf6cea793e444232b6fb25
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\04a2f4971b559bc486c518f876eb442d5aa45ba5e6c451dd33fa4581d9fbf49d
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\070efb71cea175da0e534b6108a5ee93b919c2d5f

F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\531243b0790b6237964d1436b5ce18bd0b36faa1fc008ed6fc5adbdfdcd94a33
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\532a67ea11255db3394dc10099bd9a15ab12a5e942214246f0353bed41c70914
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\543e23c1f6911026e9e895b8d0f67150cb7b9f07277446f59fcc691070f4fb4c
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\546bea8fc6370d214f6d661584552efd03a47b2bf62dafacb47ee98782149ccc
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\54df42743edcb5bee2a34a90634c15dae3debb9d737b332c47718a3b9d258f28
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\55aa62e22da5abe4a62d878373bdfb86be04f757394f4d6c8b9b253f6bc6c2ea
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\59518834f5dd5be33fb1c5e258c0b842b4e9f341c831bd1521ff43d059a480c9
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\5ad5ff3bc7a572d13c2d0c991e739df69e849011b

F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\a61271bf00789905061bd22825a89415a1306e196bcd8752ccc2b9a7264791ca
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\a6e97b13e811843814837f15db002f74b596c36ae1b6321a41f43eb24d342716
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\a87cd03c5743d52f5fb8b4a158dcefc3e7984c5fead2f486b1eca5fe7d96e3e9
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\aa5b869d5338bfdced4df959df0a8535a2e7aa6fcae28f7168d8109ff60c04cd
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\ab191f64cc74cdefc5314208c6dc6e375a519a2c7c790cfa0240630982a3a6f3
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\abe1699ae59c14c5bb5b96382775a2b56bf553ba30caf5129de22d2b76c3607b
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\ac453098d9088de21e14b23b9d6a6342f63454ee01c5b77c71ebb8ae6256d9b2
F:\BaiduNetdiskDownload\cancer_detection_homework\train\images\acdfb816ad14915969eb89e35e2f743ad3147bbb7

199

# 4.通过步骤1中文件夹字典读取ID所对应标签，存入trainy中

In [5]:
trainy = np.zeros((199))
for i in range(199):
    trainy[i] = dict[filename[i]]
print(trainy.shape)
print(trainx.shape)
print(trainy)

(199,)
(199, 640, 435)
[3. 1. 2. 4. 1. 1. 2. 1. 1. 1. 2. 2. 2. 2. 1. 4. 1. 2. 2. 2. 1. 2. 4. 2.
 1. 3. 2. 2. 2. 3. 3. 2. 1. 4. 2. 3. 1. 2. 1. 1. 1. 3. 2. 4. 1. 2. 2. 2.
 2. 4. 1. 2. 2. 3. 2. 2. 1. 2. 2. 2. 3. 1. 4. 2. 2. 3. 1. 1. 2. 2. 2. 2.
 2. 1. 3. 1. 2. 1. 1. 3. 2. 2. 1. 1. 2. 1. 1. 2. 2. 3. 3. 2. 2. 2. 2. 1.
 2. 3. 1. 3. 2. 3. 2. 1. 2. 2. 3. 1. 3. 1. 2. 2. 2. 4. 3. 2. 3. 2. 1. 3.
 2. 1. 2. 2. 3. 2. 2. 4. 1. 2. 1. 2. 1. 4. 2. 1. 1. 2. 2. 1. 2. 1. 1. 1.
 2. 3. 1. 3. 3. 2. 2. 3. 1. 1. 2. 2. 3. 3. 2. 2. 2. 3. 2. 2. 2. 2. 4. 2.
 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 1. 1. 1. 1. 2. 2. 3. 1.
 3. 2. 1. 1. 2. 1. 1.]


# 5.调整输入数据格式，使得可以传入模型中

In [6]:
trainx = trainx.reshape(199, 640, 435, 1)
trainx.shape
trainy = to_categorical(trainy, num_classes=10)

# 6.简单的CNN网络模型

In [7]:
model = Sequential()
#layer2
model.add(Conv2D(6, (3,3),strides=(1,1),input_shape = (640, 435, 1),data_format='channels_last',padding='valid',activation='relu',kernel_initializer='uniform'))
#layer3
model.add(MaxPooling2D((2,2)))
#layer4
model.add(Conv2D(16, (3,3),strides=(1,1),data_format='channels_last',padding='valid',activation='relu',kernel_initializer='uniform'))
#layer5
model.add(MaxPooling2D(2,2))
model.add(Flatten())
#layer6
model.add(Dense(200,activation='relu'))
#layer7
model.add(Dense(10,activation='softmax'))
#print
model.summary()

W0610 21:58:19.963777 12932 deprecation_wrapper.py:119] From c:\users\leo\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:4267: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 638, 433, 6)       60        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 319, 216, 6)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 317, 214, 16)      880       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 158, 107, 16)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 270496)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               54099400  
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

# 7.训练模型

In [8]:
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer=rmsprop,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print('Training ------------')

model.fit(trainx, trainy, epochs=50, batch_size=16)

print('Training ------------')

W0610 21:58:24.631161 12932 deprecation.py:323] From c:\users\leo\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Training ------------
Epoch 1/50
199/199 [==============================] - 10s 52ms/step - loss: 1.7224 - acc: 0.2814
Epoch 2/50
199/199 [==============================] - 2s 11ms/step - loss: 1.1589 - acc: 0.5176
Epoch 3/50
199/199 [==============================] - 2s 11ms/step - loss: 1.0660 - acc: 0.5779
Epoch 4/50
199/199 [==============================] - 2s 11ms/step - loss: 1.0125 - acc: 0.6080
Epoch 5/50
199/199 [==============================] - 2s 11ms/step - loss: 0.9387 - acc: 0.6281
Epoch 6/50
199/199 [==============================] - 2s 11ms/step - loss: 0.8468 - acc: 0.6533
Epoch 7/50
199/199 [==============================] - 2s 11ms/step - loss: 0.7398 - acc: 0.7236
Epoch 8/50
199/199 [==============================] - 2s 11ms/step - loss: 0.5982 - acc: 0.7688
Epoch 9/50
199/199 [==============================] - 2s 11ms/step - loss: 0.4140 - acc: 0.8392
Epoch 10/50
199/199 [==============================] - 2s 11ms/step - loss: 0.4339 - acc: 0.8191
Epoch 11/50
199/

# 8.存入模型

In [10]:
model.save('model.h5')

# 9.读取测试集步骤同上

In [31]:
filePatht = 'F:\\BaiduNetdiskDownload\\cancer_detection_homework\\test\\images' #读取相应目录下文件夹名字
filenamet = os.listdir(filePatht)
print(filenamet[10])
csv_list = pd.DataFrame(data=filenamet)
csv_list.to_csv('re.csv', encoding='gbk')

156aaebfd82ee119bf61c2a42685a5ede2bab613bbf1c4ca24e2bceaabe5e2f8


In [15]:
testx = np.zeros((87, 640, 435))
def load_data_t(data_dir):  # 
    count = 0
    for guy in os.listdir(data_dir):  
        person_dir = pjoin(data_dir, guy)  #
        for i in os.listdir(person_dir):
            image_dir = pjoin(person_dir, i)  # 
            img1 = cv2.imread(image_dir, cv2.IMREAD_GRAYSCALE)   #读取图片，第二个参数表示以灰度图像读入
            if img1 is None:                   #判断读入的img1是否为空，为空就继续下一轮循环
                continue
            res1= cv2.resize(img1,(435,640))/255              #对图片进行缩放，第一个参数是读入的图片，第二个是制定的缩放大小
            print(res1.shape)
            testx[count,:] = res1
            count = count+1
        print(person_dir)
    print(count)
    return count
load_data_t('F:\\BaiduNetdiskDownload\\cancer_detection_homework\\test\\images')

(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\0172a059fb35811fca55608ba60652429d2200a84eeddf948a26002528e39b05
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\01a9c32640ce0078472081a418c46a7d449b2615eafc146dab7bebb898bdbe7c
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\023b03b891f8f71de53cd3ad11f37b52a8d8d170e3de984b08cc8ec9e483c553
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\07986451a3c8406eba97913b2ca1798c574a51e14e95e74055d69eeb4f920e2d
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\089c3e393278d33b1ec9ecff7c0e4ca7cf553c3a9eabf7de21ff861a997977fc
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\0d83cf7541611b70c582b2e2e6e522d7b19db230cc6a657ae6958f74bf6fab09
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\0db054ed0444f81bc018e81c71501836198517b08b6bf9c05d63b757cdd65da1
(640, 435)
F:\BaiduNetdiskDownload

F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\f4f576fa01a47782f0748a8a1db5ffba1e695fc635818e46cff61fde12dc9dea
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\f8febf5faf6cb0a40123c3718ed4e749022330192ddf5f33694d9a7f7dc69e76
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\fb44d233d8e76fca02ec4c44c18d62db7035f7a4bc86fc51086b78a7bf5fcb3e
(640, 435)
F:\BaiduNetdiskDownload\cancer_detection_homework\test\images\fd816493699cccf5e2049eee419eb75d8d00c089c85000dfc11da97da5ada1e5
87


87

# 10.输出预测标签

In [22]:
testx = testx.reshape(87, 640, 435, 1)
y_pred = model.predict_classes(testx)
print (y_pred)

[2 2 1 2 2 2 2 1 1 1 2 1 2 1 1 2 2 2 4 2 2 2 2 2 2 2 2 2 3 2 2 2 2 1 1 1 2
 1 2 2 2 2 1 2 3 2 2 1 2 2 2 3 2 1 1 4 2 1 1 3 2 1 1 2 1 2 2 1 2 2 2 1 2 3
 2 2 1 2 2 2 1 2 2 2 2 2 1]


In [32]:
y_pred.shape

(87,)

# 11.存入数据

In [33]:
np.savetxt('new.csv', y_pred, delimiter = ' ')